In [1]:
import pandas as pd
from datetime import date, datetime

## Tạo Bảng mối quan hệ các Chỉ số - DIM ACCOUNT

In [2]:
# create dim_account_relation
stock_relation = pd.read_excel('./stock_relation.xlsx', sheet_name='dim_account')
account_relation = pd.read_excel('./stock_relation.xlsx', sheet_name='account_relation')
stock_relation

,account_key,account_name,sign,account_type,level
0,270,TỔNG TÀI SẢN,1,cdkt,1
1,100,TÀI SẢN NGẮN HẠN,1,cdkt,2
2,110,Tiền và tương đương tiền,1,cdkt,3
3,111,Tiền,1,cdkt,4
4,112,Các khoản tương đương tiền,1,cdkt,4
...,...,...,...,...,...
134,62,Lợi ích của cổ đông thiểu số,1,kqkd,1
135,61,Lợi nhuận của Cổ đông của Công ty mẹ,1,kqkd,1
136,70,Lãi cơ bản trên cổ phiếu,1,kqkd,1
137,71,Lãi trên cổ phiếu pha loãng,1,kqkd,1


### Mối quan hệ giữa các chỉ số trong Bảng CĐKT

In [3]:
dim_bs_relation = stock_relation[stock_relation["level"] == 4]

# Add Account column
dim_bs_relation = pd.merge(dim_bs_relation, account_relation, left_on='account_key', right_on='child', how='left')
dim_bs_relation['account_id_lv3'] = dim_bs_relation['parent']
dim_bs_relation = dim_bs_relation.drop(columns=['parent', 'child', 'level'])
merged_df = pd.merge(dim_bs_relation['account_id_lv3'], stock_relation, left_on='account_id_lv3', right_on='account_key', how='left')
dim_bs_relation["Account"] = merged_df["account_name"]

# Add Sub Class column
dim_bs_relation = pd.merge(dim_bs_relation, account_relation, left_on='account_id_lv3', right_on='child', how='left')
dim_bs_relation['account_id_lv2'] = dim_bs_relation['parent']
dim_bs_relation = dim_bs_relation.drop(columns=['parent', 'child'])
merged_df = pd.merge(dim_bs_relation['account_id_lv2'], stock_relation, left_on='account_id_lv2', right_on='account_key', how='left')
dim_bs_relation["Sub Class"] = merged_df["account_name"]

# Add Class column
dim_bs_relation = pd.merge(dim_bs_relation, account_relation, left_on='account_id_lv2', right_on='child', how='left')
dim_bs_relation['account_id_lv1'] = dim_bs_relation['parent']
dim_bs_relation = dim_bs_relation.drop(columns=['parent', 'child'])
merged_df = pd.merge(dim_bs_relation['account_id_lv1'], stock_relation, left_on='account_id_lv1', right_on='account_key', how='left')
dim_bs_relation["Class"] = merged_df["account_name"]

dim_bs_relation

,account_key,account_name,sign,account_type,account_id_lv3,Account,account_id_lv2,Sub Class,account_id_lv1,Class
0,111,Tiền,1,cdkt,110,Tiền và tương đương tiền,100,TÀI SẢN NGẮN HẠN,270,TỔNG TÀI SẢN
1,112,Các khoản tương đương tiền,1,cdkt,110,Tiền và tương đương tiền,100,TÀI SẢN NGẮN HẠN,270,TỔNG TÀI SẢN
2,121,Đầu tư ngắn hạn,1,cdkt,120,Giá trị thuần đầu tư ngắn hạn,100,TÀI SẢN NGẮN HẠN,270,TỔNG TÀI SẢN
3,122,Dự phòng đầu tư ngắn hạn,-1,cdkt,120,Giá trị thuần đầu tư ngắn hạn,100,TÀI SẢN NGẮN HẠN,270,TỔNG TÀI SẢN
4,123,Chứng khoán đầu tư giữ đến ngày đáo hạn,1,cdkt,120,Giá trị thuần đầu tư ngắn hạn,100,TÀI SẢN NGẮN HẠN,270,TỔNG TÀI SẢN
...,...,...,...,...,...,...,...,...,...,...
78,420,Quỹ khác,1,cdkt,410,Vốn và các quỹ,400,VỐN CHỦ SỞ HỮU,440,TỔNG CỘNG NGUỒN VỐN
79,421,Lãi chưa phân phối,1,cdkt,410,Vốn và các quỹ,400,VỐN CHỦ SỞ HỮU,440,TỔNG CỘNG NGUỒN VỐN
80,429,Lợi ích cổ đông không kiểm soát,1,cdkt,410,Vốn và các quỹ,400,VỐN CHỦ SỞ HỮU,440,TỔNG CỘNG NGUỒN VỐN
81,431,Vốn ngân sách nhà nước,1,cdkt,430,Vốn Ngân sách nhà nước và quỹ khác,400,VỐN CHỦ SỞ HỮU,440,TỔNG CỘNG NGUỒN VỐN


In [18]:
dim_is_relation = stock_relation[stock_relation["account_type"] == "kqkd"]

### Chuẩn bị dữ liệu Bảng CĐKT, KQKD


In [4]:
stocks = ['HPG', 'HSG', 'NKG', 'TVN', 'TLH']

In [5]:
# Hàm chuyển đổi từ Quý sang ngày đầu tháng

def convert_quarter_to_date(quarter_str):
    quarter, year = quarter_str.split()
    year = int(float(year))
    if quarter == 'Q1':
        return f"{year}-01-01"
    elif quarter == 'Q2':
        return f"{year}-04-01"
    elif quarter == 'Q3':
        return f"{year}-07-01"
    elif quarter == 'Q4':
        return f"{year}-10-01"
    else:
        raise ValueError(f"Invalid quarter: {quarter}")

In [6]:
hpg = pd.read_excel('./Raw Data/HSG_BS.xlsx', skiprows=7).dropna()


In [7]:
hpg = pd.read_excel('./Raw Data/HPG_BS.xlsx', skiprows=7).dropna()

# Tạo cột account_key cho bảng cdkt
hpg = pd.merge(hpg, stock_relation, left_on='CHỈ TIÊU', right_on='account_name', how='left')
hpg = hpg.drop(columns=['CHỈ TIÊU', 'account_name', 'sign', 'account_type', 'level'])
hpg = hpg.melt(id_vars=['account_key'], var_name='quarter', value_name='End of Period')
hpg['date'] = hpg['quarter'].apply(convert_quarter_to_date)
hpg = hpg.drop(columns=['quarter'])
hpg['stock_name'] = 'HPG'
hpg

,account_key,End of Period,date,stock_name
0,270,1.174723e+14,2020-07-01,HPG
1,100,4.331936e+13,2020-07-01,HPG
2,110,5.628549e+12,2020-07-01,HPG
3,111,2.014169e+12,2020-07-01,HPG
4,112,3.614380e+12,2020-07-01,HPG
...,...,...,...,...
1720,429,1.041126e+11,2024-01-01,HPG
1721,430,0.000000e+00,2024-01-01,HPG
1722,431,0.000000e+00,2024-01-01,HPG
1723,432,0.000000e+00,2024-01-01,HPG


In [8]:
def transform_report(stock, report_type):
    df = pd.read_excel(f'./Raw Data/{stock}_{report_type}.xlsx', skiprows=7).dropna()

    if report_type == 'IS':
        df = df[:-2]

    df = pd.merge(df, stock_relation, left_on='CHỈ TIÊU', right_on='account_name', how='left')
    df = df.drop(columns=['CHỈ TIÊU', 'account_name', 'sign', 'account_type', 'level'])
    df = df.melt(id_vars=['account_key'], var_name='quarter', value_name='End of Period')
    df['date'] = df['quarter'].apply(convert_quarter_to_date)
    df = df.drop(columns=['quarter'])
    df['stock_name'] = stock

    return df

### Chuẩn bị dữ liệu Giá cổ Phiếu

In [9]:
from vnstock import *

In [10]:
# Hàm lấy giá cố phiếu
def get_stock_price(stock):
    stock_hist = stock_historical_data(stock, "2020-07-01",
        str(date.today()), "1D", "stock")
    stock_hist["%_change"] = round(((stock_hist['close'] - stock_hist['close'].shift(1))
                                    / stock_hist['close'].shift(1)) * 100, 2)
    stock_hist['change'] = (stock_hist['close'] - stock_hist['close'].shift(1))
    stock_hist['time'] = stock_hist['time'].astype(str)
    stock_hist = stock_hist.rename(columns={'ticker': 'stock_name', 'time': 'date'})

    return stock_hist
    

In [11]:
get_stock_price('HPG')

,date,open,high,low,close,volume,stock_name,%_change,change
0,2020-07-01,11180,11380,11140,11350,10843030,HPG,NaN,NaN
1,2020-07-02,11380,11490,11350,11430,13000820,HPG,0.70,80.0
2,2020-07-03,11470,11490,11350,11380,8111500,HPG,-0.44,-50.0
3,2020-07-06,11470,11490,11360,11430,9764620,HPG,0.44,50.0
4,2020-07-07,11490,11960,11470,11740,28048770,HPG,2.71,310.0
...,...,...,...,...,...,...,...,...,...
1017,2024-07-24,27750,27950,27100,27700,19233100,HPG,-0.36,-100.0
1018,2024-07-25,27600,27650,27300,27350,13319700,HPG,-1.26,-350.0
1019,2024-07-26,27350,27650,27350,27450,8806900,HPG,0.37,100.0
1020,2024-07-29,27650,28100,27600,27900,15443700,HPG,1.64,450.0


### Chuẩn bị thông tin cơ bản về cổ phiếu

In [12]:
def get_stock_fundamental_info(stock):
    fundamental_ratio = company_fundamental_ratio(stock) 
    profile = company_profile(stock)
    overview = company_overview(stock)
    evaluation = stock_evaluation(stock)[["ticker", "PE", "industryPE", "PB", "industryPB"]][-1:]

    stock_info = pd.merge(fundamental_ratio[['ticker']], profile[['companyName', 'ticker']], on='ticker', how='inner')
    stock_info = pd.merge(stock_info, overview[['ticker', 'outstandingShare', 'exchange']], on='ticker', how='inner')
    stock_info = pd.merge(stock_info, evaluation, on='ticker', how='inner')

    stock_info = stock_info.rename(columns={'ticker': 'stock_name', 'companyName': 'company_name', 'outstandingShare': 'outstanding_share'})
    stock_info['outstanding_share'] = stock_info['outstanding_share'] * 1000000

    return stock_info

In [13]:
get_stock_fundamental_info('HPG')

,stock_name,company_name,outstanding_share,exchange,PE,industryPE,PB,industryPB
0,HPG,Công ty Cổ phần Tập đoàn Hòa Phát,6.396300e+09,HOSE,19.2,27.0,1.7,1.4


### Tổng hợp tất cả các DataFrame

In [14]:
# Tổng hợp dữ liệu của tất cả các cổ phiếu 
bs_list = [] 
is_list = []
stock_price_list = []
stock_info_list = []

for stock in stocks:
    balance_sheet = transform_report(stock, 'BS')
    income_statement = transform_report(stock, 'IS')
    stock_price = get_stock_price(stock)
    stock_info = get_stock_fundamental_info(stock)
    
    bs_list.append(balance_sheet)
    is_list.append(income_statement)
    stock_price_list.append(stock_price)
    stock_info_list.append(stock_info)

bs_df = pd.concat(bs_list, ignore_index=True)
is_df = pd.concat(is_list, ignore_index=True)
stock_price_df = pd.concat(stock_price_list, ignore_index=True)
stock_info_df = pd.concat(stock_info_list, ignore_index=True)

bs_df

,account_key,End of Period,date,stock_name
0,270,1.174723e+14,2020-07-01,HPG
1,100,4.331936e+13,2020-07-01,HPG
2,110,5.628549e+12,2020-07-01,HPG
3,111,2.014169e+12,2020-07-01,HPG
4,112,3.614380e+12,2020-07-01,HPG
...,...,...,...,...
8620,429,2.719653e+10,2024-01-01,TLH
8621,430,0.000000e+00,2024-01-01,TLH
8622,431,0.000000e+00,2024-01-01,TLH
8623,432,0.000000e+00,2024-01-01,TLH


In [15]:
stock_info_df

,stock_name,company_name,outstanding_share,exchange,PE,industryPE,PB,industryPB
0,HPG,Công ty Cổ phần Tập đoàn Hòa Phát,6.396300e+09,HOSE,19.2,27.0,1.7,1.4
1,HSG,Công ty Cổ phần Tập đoàn Hoa Sen,6.160000e+08,HOSE,12.6,27.0,1.3,1.4
2,NKG,Công ty Cổ phần Thép Nam Kim,2.633000e+08,HOSE,19.3,27.0,1.1,1.4
3,TVN,Tổng Công ty Thép Việt Nam - Công ty Cổ phần,6.780000e+08,UPCOM,-14.3,27.0,0.8,1.4
4,TLH,Công ty Cổ phần Tập đoàn Thép Tiến Lên,1.123000e+08,HOSE,-576.2,27.0,0.5,1.4


In [16]:
stock_price_df

,date,open,high,low,close,volume,stock_name,%_change,change
0,2020-07-01,11180,11380,11140,11350,10843030,HPG,NaN,NaN
1,2020-07-02,11380,11490,11350,11430,13000820,HPG,0.70,80.0
2,2020-07-03,11470,11490,11350,11380,8111500,HPG,-0.44,-50.0
3,2020-07-06,11470,11490,11360,11430,9764620,HPG,0.44,50.0
4,2020-07-07,11490,11960,11470,11740,28048770,HPG,2.71,310.0
...,...,...,...,...,...,...,...,...,...
5105,2024-07-24,7230,7350,7100,7280,1585800,TLH,0.41,30.0
5106,2024-07-25,7250,7310,7220,7250,602100,TLH,-0.41,-30.0
5107,2024-07-26,7300,7380,7230,7380,629800,TLH,1.79,130.0
5108,2024-07-29,7500,7510,7350,7470,866600,TLH,1.22,90.0


## Chuyển dữ liệu vào file Excel

In [19]:
with pd.ExcelWriter('./Processed Data/data.xlsx') as writer:
    dim_bs_relation.to_excel(writer, sheet_name='dim_bs_relation', index=False)
    dim_is_relation.to_excel(writer, sheet_name='dim_is_relation', index=False)
    bs_df.to_excel(writer, sheet_name='cdkt', index=False)
    is_df.to_excel(writer, sheet_name='kqkd', index=False)
    stock_price_df.to_excel(writer, sheet_name='stock_price', index=False)
    stock_info_df.to_excel(writer, sheet_name='stock_info', index=False)